<a href="https://colab.research.google.com/github/spaziochirale/EsperimentiVari/blob/main/rag_chatbot_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prototipo di ChatBot RAG con LangChain e PDF

Questo notebook mostra come realizzare un *chatBot* di tipo **RAG** utilizzando il framework **LangChain** e un file PDF come base di conoscenza.

Per prima cosa installiamo sul server alcuni package Python della piattaforma LangChain e altre librerie necessarie che utilizzeremo nel prototipo.

In [ ]:
!pip install -qU langchain langchain_community langchain_chroma pypdf langchain-openai

Impostiamo la chiave API di OpenAI e creiamo l'oggetto `llm`.

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

Importiamo le librerie necessarie per il nostro progetto.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

La cella che segue genera una lista di oggetti di tipo ***LangChain Document*** a partire dal file PDF specificato come percorso. Un Document LangChain ha come contenuto un testo in formato TXT.
**PyPDFLoader** crea un documento per ogni pagina del file PDF.

Le istruzioni commentate consentono di caricare il file attraverso l'utility Colab ***files.upload***, tuttavia è più veloce trascinare il file nell'area File System del Notebook attivo e poi copiare direttamente il percorso come argomento stringa alla PyPDFLoader.

In [ ]:
#from google.colab import files
#uploaded = files.upload()
#pdf_file = list(uploaded.keys())[0]  # Prende il nome del primo file caricato

# AGGIORNARE CON IL PATH EFFETTIVO AL FILE CARICATO
loader = PyPDFLoader('/content/Z9RG_(It)05.pdf')
docs = loader.load()
print('Numero pagine caricate:',len(docs))

Dividiamo il contenuto del PDF in chunks e creiamo il database vettoriale. Per un documento PDF ben strutturato ogni pagina potrebbe corrispondere ad un chunk. Il parametro chunk_size è sovradimensionato rispetto alla lunghezza in caratteri della pagina di un tipico manuale utente PDF, per cui il numero di chunks dovrebbe essere pari al numero di pagine per la maggior parte dei documenti.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=6000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
print('Numero chunks creati:',len(splits))

# Creo il vectorstore utilizzando la versione con rappresentazione in memoria di Chroma (non viene creato un db su file)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(model="text-embedding-3-large"))

Creiamo il retriever e prepariamo il prompt template.

In [ ]:
retriever = vectorstore.as_retriever()

template = """Sei un assistente che aiuta l'utente a trovare soluzioni alle procedure di uso del prodotto. Usa come contesto i contenuti recuperati dal manuale utente. Per la risposta usa anche le tue competenze generali, ma se il contesto non contiene elementi pertinenti consiglia di rivolgersi ad un assistente umano.

Question: {question}
Context: {context}"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)


Definiamo una funzione di utilità per formattare i documenti.

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

Definiamo la chain RAG.

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

Ora possiamo utilizzare la chain RAG per porre domande sul contenuto del PDF.

In [ ]:
question = "Perché il monitor nel mirino è spento?"
rag_chain.invoke(question)

Se vogliamo testare l'operazione di retrival, possiamo usare la cella seguente.

In [ ]:
retriever.invoke("Perché il monitor nel mirino è spento?")